In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [32]:
Data=pd.read_csv('/kaggle/input/dataset-for-bank-loan-prediction/credit_train.csv')
print('length of Data',len(Data))
Data.head()

length of Data 100514


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0


In [33]:
round((Data.isna().sum()/len(Data))*100,2)

Loan ID                          0.51
Customer ID                      0.51
Loan Status                      0.51
Current Loan Amount              0.51
Term                             0.51
Credit Score                    19.57
Annual Income                   19.57
Years in current job             4.71
Home Ownership                   0.51
Purpose                          0.51
Monthly Debt                     0.51
Years of Credit History          0.51
Months since last delinquent    53.38
Number of Open Accounts          0.51
Number of Credit Problems        0.51
Current Credit Balance           0.51
Maximum Open Credit              0.51
Bankruptcies                     0.71
Tax Liens                        0.52
dtype: float64

# **Analyzing column values to discretize data into numbers**

In [34]:
for column in Data.columns:
    print(column,'->',Data[column].unique(),'->',round(len(Data[column].unique())/1005,2),'%',end='\n\n\n')

Loan ID -> ['14dd8831-6af5-400b-83ec-68e61888a048'
 '4771cc26-131a-45db-b5aa-537ea4ba5342'
 '4eed4e6a-aa2f-4c91-8651-ce984ee8fb26' ...
 '81ab928b-d1a5-4523-9a3c-271ebb01b4fb'
 'c63916c6-6d46-47a9-949a-51d09af4414f' nan] -> 81.59 %


Customer ID -> ['981165ec-3274-42f5-a3b4-d104041a9ca9'
 '2de017a3-2e01-49cb-a581-08169e83be29'
 '5efb2b2b-bf11-4dfd-a572-3761a2694725' ...
 '3e45ffda-99fd-4cfc-b8b8-446f4a505f36'
 '1b3014be-5c07-4d41-abe7-44573c375886' nan] -> 81.59 %


Loan Status -> ['Fully Paid' 'Charged Off' nan] -> 0.0 %


Current Loan Amount -> [  445412.   262328. 99999999. ...   100254.   274076.       nan] -> 21.9 %


Term -> ['Short Term' 'Long Term' nan] -> 0.0 %


Credit Score -> [ 709.   nan  741.  721. 7290.  730.  678.  739.  728.  740.  743.  727.
  723.  747.  687.  750.  714.  724.  704.  688.  749.  746.  737.  729.
  733.  725.  745.  720.  718.  682. 7120.  680.  710.  598.  719. 6610.
  652.  736. 7380.  644.  672. 7370.  699.  751.  694.  675.  657.  748.
  666.  734.

# Gathered Important features for predictions include:

* Loan Status
* Current loan Amount
* Term
* Credit Score
* Annual income
* Years in current job
* Home Ownership
* Purpose
* Monthly dept
* Number of open accounts
* current credit balance
* bankruptcies
* Tax Liens

In [35]:
Data.drop(['Loan ID','Customer ID','Number of Credit Problems','Months since last delinquent','Years of Credit History','Maximum Open Credit'],axis=1,inplace=True)

In [36]:
Data.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Number of Open Accounts,Current Credit Balance,Bankruptcies,Tax Liens
0,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,6.0,228190.0,1.0,0.0
1,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,35.0,229976.0,0.0,0.0
2,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,18.0,297996.0,0.0,0.0
3,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,9.0,256329.0,0.0,0.0
4,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,15.0,253460.0,0.0,0.0
5,Charged Off,206602.0,Short Term,7290.0,896857.0,10+ years,Home Mortgage,Debt Consolidation,16367.74,6.0,215308.0,0.0,0.0
6,Fully Paid,217646.0,Short Term,730.0,1184194.0,< 1 year,Home Mortgage,Debt Consolidation,10855.08,13.0,122170.0,1.0,0.0
7,Charged Off,648714.0,Long Term,NaN,NaN,< 1 year,Home Mortgage,Buy House,14806.13,15.0,193306.0,0.0,0.0
8,Fully Paid,548746.0,Short Term,678.0,2559110.0,2 years,Rent,Debt Consolidation,18660.28,4.0,437171.0,0.0,0.0
9,Fully Paid,215952.0,Short Term,739.0,1454735.0,< 1 year,Rent,Debt Consolidation,39277.75,20.0,669560.0,0.0,0.0


# Encoding Columns string to numerical values

* Loan Status
* Term
* Years in current job
* Home Ownership
* Purpose

In [37]:
Data['Loan Status']=Data['Loan Status'].replace({'Fully Paid':1, 'Charged Off':0})
Data['Term']=Data['Term'].replace({'Short Term':1 ,'Long Term':2})
Data['Years in current job']=Data['Years in current job'].replace({'8 years':8, '10+ years':11 ,'3 years':3 ,'5 years':5 ,'< 1 year':1,'2 years':2 ,'4 years':4 ,'9 years':9 ,'7 years':7 ,'1 year':1 ,'6 years':6})
Data['Home Ownership']=Data['Home Ownership'].replace({'Home Mortgage':1 ,'Own Home':2 ,'Rent':3 ,'HaveMortgage':4})
Data['Purpose']=Data['Purpose'].replace({'Home Improvements':1 ,'Debt Consolidation':2 ,'Buy House':3 ,'other':4 ,'Business Loan':5 ,'Buy a Car':6 ,'major_purchase':7 ,'Take a Trip':8 ,'Other':9 ,'small_business':10 ,'Medical Bills':11 ,'wedding':12 ,'vacation':13 ,'Educational Expenses':14 ,'moving':15 ,'renewable_energy':16})

/tmp/ipykernel_35/521676003.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Data['Loan Status']=Data['Loan Status'].replace({'Fully Paid':1, 'Charged Off':0})
/tmp/ipykernel_35/521676003.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Data['Term']=Data['Term'].replace({'Short Term':1 ,'Long Term':2})
/tmp/ipykernel_35/521676003.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-i

In [38]:
Data.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Number of Open Accounts,Current Credit Balance,Bankruptcies,Tax Liens
0,1.0,445412.0,1.0,709.0,1167493.0,8.0,1.0,1.0,5214.74,6.0,228190.0,1.0,0.0
1,1.0,262328.0,1.0,NaN,NaN,11.0,1.0,2.0,33295.98,35.0,229976.0,0.0,0.0
2,1.0,99999999.0,1.0,741.0,2231892.0,8.0,2.0,2.0,29200.53,18.0,297996.0,0.0,0.0
3,1.0,347666.0,2.0,721.0,806949.0,3.0,2.0,2.0,8741.90,9.0,256329.0,0.0,0.0
4,1.0,176220.0,1.0,NaN,NaN,5.0,3.0,2.0,20639.70,15.0,253460.0,0.0,0.0
5,0.0,206602.0,1.0,7290.0,896857.0,11.0,1.0,2.0,16367.74,6.0,215308.0,0.0,0.0
6,1.0,217646.0,1.0,730.0,1184194.0,1.0,1.0,2.0,10855.08,13.0,122170.0,1.0,0.0
7,0.0,648714.0,2.0,NaN,NaN,1.0,1.0,3.0,14806.13,15.0,193306.0,0.0,0.0
8,1.0,548746.0,1.0,678.0,2559110.0,2.0,3.0,2.0,18660.28,4.0,437171.0,0.0,0.0
9,1.0,215952.0,1.0,739.0,1454735.0,1.0,3.0,2.0,39277.75,20.0,669560.0,0.0,0.0


# Removing Outliers

In [39]:
for col in ['Current Loan Amount','Credit Score','Annual Income','Monthly Debt','Current Credit Balance','Bankruptcies','Tax Liens']:
    Q1 = Data[col].quantile(0.25)
    Q3 = Data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    Data[col] = Data[col].where((Data[col] >= lower) & (Data[col] <= upper), np.nan)

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/co

# Removing NaN values

In [40]:
Data.dropna(inplace=True)

In [41]:
print('Length of filtered Data:',len(Data))
Data.head()

Length of filtered Data: 46720


,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Number of Open Accounts,Current Credit Balance,Bankruptcies,Tax Liens
3,1.0,347666.0,2.0,721.0,806949.0,3.0,2.0,2.0,8741.90,9.0,256329.0,0.0,0.0
8,1.0,548746.0,1.0,678.0,2559110.0,2.0,3.0,2.0,18660.28,4.0,437171.0,0.0,0.0
9,1.0,215952.0,1.0,739.0,1454735.0,1.0,3.0,2.0,39277.75,20.0,669560.0,0.0,0.0
19,1.0,390390.0,1.0,747.0,1791738.0,8.0,1.0,1.0,2478.55,6.0,121182.0,0.0,0.0
20,0.0,317108.0,2.0,687.0,1133274.0,8.0,3.0,2.0,9632.81,4.0,60287.0,0.0,0.0


# Normalizing large values

In [42]:
for col in ['Current Loan Amount','Credit Score','Annual Income','Monthly Debt','Current Credit Balance']:
    Data[col]=Data[col]/(Data[col].mean()+Data[col].min())/2

In [43]:
Data.head(10)

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Number of Open Accounts,Current Credit Balance,Bankruptcies,Tax Liens
3,1.0,0.554398,2.0,0.263189,0.311900,3.0,2.0,2.0,0.262192,9.0,0.523360,0.0,0.0
8,1.0,0.875046,1.0,0.247492,0.989140,2.0,3.0,2.0,0.559670,4.0,0.892594,0.0,0.0
9,1.0,0.344363,1.0,0.269759,0.562280,1.0,3.0,2.0,1.178042,20.0,1.367074,0.0,0.0
19,1.0,0.622527,1.0,0.272680,0.692538,8.0,1.0,1.0,0.074338,6.0,0.247423,0.0,0.0
20,0.0,0.505669,2.0,0.250778,0.438030,8.0,3.0,2.0,0.288913,4.0,0.123091,0.0,0.0
21,1.0,0.204492,1.0,0.273775,0.523372,1.0,3.0,2.0,0.395967,7.0,0.269380,0.0,0.0
23,1.0,0.146537,1.0,0.264284,0.328688,11.0,1.0,2.0,0.175779,7.0,0.195208,0.0,0.0
25,1.0,0.742156,2.0,0.251143,0.665835,3.0,3.0,3.0,0.469308,7.0,0.219609,0.0,0.0
27,1.0,0.707951,1.0,0.273410,0.553644,2.0,1.0,2.0,0.755400,15.0,0.699248,0.0,0.0
31,1.0,0.533594,1.0,0.266109,0.521265,2.0,3.0,2.0,0.507294,16.0,0.639429,0.0,0.0


# Designing Model

In [44]:
Model=tf.keras.Sequential([
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [45]:
Model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Gathering X, Y from Filtered Dataset

In [46]:
Y=Data['Loan Status']
X=Data.drop(['Loan Status'],axis=1)

In [47]:
X.head()

,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Number of Open Accounts,Current Credit Balance,Bankruptcies,Tax Liens
3,0.554398,2.0,0.263189,0.311900,3.0,2.0,2.0,0.262192,9.0,0.523360,0.0,0.0
8,0.875046,1.0,0.247492,0.989140,2.0,3.0,2.0,0.559670,4.0,0.892594,0.0,0.0
9,0.344363,1.0,0.269759,0.562280,1.0,3.0,2.0,1.178042,20.0,1.367074,0.0,0.0
19,0.622527,1.0,0.272680,0.692538,8.0,1.0,1.0,0.074338,6.0,0.247423,0.0,0.0
20,0.505669,2.0,0.250778,0.438030,8.0,3.0,2.0,0.288913,4.0,0.123091,0.0,0.0


In [48]:
Y

3        1.0
8        1.0
9        1.0
19       1.0
20       0.0
        ... 
99985    1.0
99987    1.0
99994    1.0
99995    1.0
99998    1.0
Name: Loan Status, Length: 46720, dtype: float64

# Training Model

In [49]:
Model.fit(X,Y,batch_size=32,epochs=10)

Epoch 1/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8073 - loss: 0.5017
Epoch 2/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8101 - loss: 0.4851
Epoch 3/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8033 - loss: 0.4904
Epoch 4/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8095 - loss: 0.4806
Epoch 5/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8058 - loss: 0.4849
Epoch 6/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8111 - loss: 0.4759
Epoch 7/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8086 - loss: 0.4795
Epoch 8/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8070 - loss: 0.4811
Epoch 9/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8079 - loss: 0.4788
Epoch 10/10
1460/1460 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8069 - loss: 0.4807


# Validating Model

In [50]:
TestData=pd.read_csv('/kaggle/input/dataset-for-bank-loan-prediction/credit_test.csv')
TestData.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f738779f-c726-40dc-92cf-689d73af533d,ded0b3c3-6bf4-4091-8726-47039f2c1b90,611314.0,Short Term,747.0,2074116.0,10+ years,Home Mortgage,Debt Consolidation,42000.83,21.8,NaN,9.0,0.0,621908.0,1058970.0,0.0,0.0
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,1630e6e3-34e3-461a-8fda-09297d3140c8,266662.0,Short Term,734.0,1919190.0,10+ years,Home Mortgage,Debt Consolidation,36624.40,19.4,NaN,11.0,0.0,679573.0,904442.0,0.0,0.0
2,f7744d01-894b-49c3-8777-fc6431a2cff1,2c60938b-ad2b-4702-804d-eeca43949c52,153494.0,Short Term,709.0,871112.0,2 years,Rent,Debt Consolidation,8391.73,12.5,10.0,10.0,0.0,38532.0,388036.0,0.0,0.0
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,12116614-2f3c-4d16-ad34-d92883718806,176242.0,Short Term,727.0,780083.0,10+ years,Rent,Debt Consolidation,16771.87,16.5,27.0,16.0,1.0,156940.0,531322.0,1.0,0.0
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,39888105-fd5f-4023-860a-30a3e6f5ccb7,321992.0,Short Term,744.0,1761148.0,10+ years,Home Mortgage,Debt Consolidation,39478.77,26.0,44.0,14.0,0.0,359765.0,468072.0,0.0,0.0


In [51]:
TestData['Term']=TestData['Term'].replace({'Short Term':1 ,'Long Term':2})
TestData['Years in current job']=TestData['Years in current job'].replace({'8 years':8, '10+ years':11 ,'3 years':3 ,'5 years':5 ,'< 1 year':1,'2 years':2 ,'4 years':4 ,'9 years':9 ,'7 years':7 ,'1 year':1 ,'6 years':6})
TestData['Home Ownership']=TestData['Home Ownership'].replace({'Home Mortgage':1 ,'Own Home':2 ,'Rent':3 ,'HaveMortgage':4})
TestData['Purpose']=TestData['Purpose'].replace({'Home Improvements':1 ,'Debt Consolidation':2 ,'Buy House':3 ,'other':4 ,'Business Loan':5 ,'Buy a Car':6 ,'major_purchase':7 ,'Take a Trip':8 ,'Other':9 ,'small_business':10 ,'Medical Bills':11 ,'wedding':12 ,'vacation':13 ,'Educational Expenses':14 ,'moving':15 ,'renewable_energy':16})

/tmp/ipykernel_35/729062061.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  TestData['Term']=TestData['Term'].replace({'Short Term':1 ,'Long Term':2})
/tmp/ipykernel_35/729062061.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  TestData['Years in current job']=TestData['Years in current job'].replace({'8 years':8, '10+ years':11 ,'3 years':3 ,'5 years':5 ,'< 1 year':1,'2 years':2 ,'4 years':4 ,'9 years':9 ,'7 years':7 ,'1 year':1 ,'6 years':6})
/tmp/ipykernel_35/729062061.py:3: FutureWarning: Downcasting behavior in `replac

In [52]:
TestData.drop(['Loan ID','Customer ID','Number of Credit Problems','Months since last delinquent','Years of Credit History','Maximum Open Credit'],axis=1,inplace=True)
for col in ['Current Loan Amount','Credit Score','Annual Income','Monthly Debt','Current Credit Balance','Bankruptcies','Tax Liens']:
    Q1 = TestData[col].quantile(0.25)
    Q3 = TestData[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    TestData[col] = TestData[col].where((TestData[col] >= lower) & (TestData[col] <= upper), np.nan)

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/co

In [53]:
TestData.dropna(inplace=True)

for col in ['Current Loan Amount','Credit Score','Annual Income','Monthly Debt','Current Credit Balance']:
    TestData[col]=TestData[col]/(TestData[col].mean()+TestData[col].min())/2

In [54]:
print('Length of filtered TestData:',len(TestData))
TestData.head()

Length of filtered TestData: 4659


,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Number of Open Accounts,Current Credit Balance,Bankruptcies,Tax Liens
0,0.958682,1.0,0.271986,0.793066,11.0,1.0,2.0,1.249057,9.0,1.279134,0.0,0.0
1,0.418188,1.0,0.267253,0.733828,11.0,1.0,2.0,1.089168,11.0,1.397739,0.0,0.0
2,0.240714,1.0,0.258150,0.333081,2.0,3.0,2.0,0.249561,10.0,0.079252,0.0,0.0
4,0.504958,1.0,0.270894,0.673398,11.0,1.0,2.0,1.174054,14.0,0.739961,0.0,0.0
5,0.318238,1.0,0.269801,0.290741,1.0,3.0,2.0,0.194096,6.0,0.531982,0.0,0.0


In [55]:
predictions=Model.predict(TestData)

146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [56]:
predictions=(predictions > 0.5).astype(int)

In [59]:
predictions[-200:]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    